In [0]:
catalog_name = "dmc_tallerfinaldatabricks_mariovento"

schema_bronze = "bronze"
schema_silver = "silver"

In [0]:
from pyspark.sql.functions import col, initcap, trim, when, to_date

In [0]:
silver_devoluciones = (
  spark.table(f"{catalog_name}.{schema_bronze}.devoluciones_raw")
  # Convierte 'id_devolucion' a tipo bigint
  .withColumn("id_devolucion", col("id_devolucion").cast("bigint"))
  # Convierte 'id_venta' a tipo bigint
  .withColumn("id_venta", col("id_venta").cast("bigint"))
  # Limpia y normaliza la columna 'motivo'
  .withColumn(
    "motivo",
    when(
      (trim(col("motivo")) == "N/A") | (trim(col("motivo")) == "null") | (col("motivo").isNull()),
      "Desconocido"
    ).otherwise(initcap(trim(col("motivo")))).cast("string")
  )
  # Normaliza y convierte la columna 'fecha_devolucion' a tipo timestamp
  .withColumn("fecha_devolucion",
        when(col("fecha_devolucion") == "N/A", None)
        .when(col("fecha_devolucion").rlike("^[0-9]{4}-[0-9]{2}-[0-9]{2}$"), to_date(col("fecha_devolucion"), "yyyy-MM-dd").cast("timestamp"))
        .when(col("fecha_devolucion").rlike("^[0-9]{2}-[0-9]{2}-[0-9]{4}$"), to_date(col("fecha_devolucion"), "MM-dd-yyyy").cast("timestamp"))
        .when(col("fecha_devolucion").rlike("^[0-9]{2}/[0-9]{2}/[0-9]{4}$"), to_date(col("fecha_devolucion"), "dd/MM/yyyy").cast("timestamp"))
        .when(col("fecha_devolucion").rlike("^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2}$"), to_date(col("fecha_devolucion"), "yyyy/MM/dd HH:mm:ss").cast("timestamp"))  
        .otherwise(None)
    )
  # Elimina duplicados basados en 'id_devolucion'
  .dropDuplicates(["id_devolucion"])
)

In [0]:
display(silver_devoluciones.select("motivo").distinct())

In [0]:
display(silver_devoluciones)

In [0]:
silver_devoluciones.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_silver}.devoluciones")